### Hw-2: Task 2
### Taslima Akter
### ID: takter

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt
from operator import itemgetter
from sklearn.datasets import fetch_mldata

from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical

%matplotlib notebook

Using TensorFlow backend.


In [2]:
config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.33

### Read file names for training data

In [3]:
### File names for Training Data

import glob
import librosa
fname_trn=[]
for filename in glob.glob('/opt/e533/timit-homework/tr/trn*.wav'):
    (fname_trn.append(filename))
fname_trn.sort()
# fname_trn

fname_trs=[]
for filename in glob.glob('/opt/e533/timit-homework/tr/trs*.wav'):
    (fname_trs.append(filename))
fname_trs.sort()
# fname_trs

fname_trx=[]
for filename in glob.glob('/opt/e533/timit-homework/tr/trx*.wav'):
    (fname_trx.append(filename))
fname_trx.sort()
fname_trx[100]


'/opt/e533/timit-homework/tr/trx0100.wav'

### Read file names for validation data

In [4]:
### File names for validation Data

fname_val_n=[]
for filename in glob.glob('/opt/e533/timit-homework/v/vn*.wav'):
    (fname_val_n.append(filename))
fname_val_n.sort()
# print(fname_val_n)

fname_val_s=[]
for filename in glob.glob('/opt/e533/timit-homework/v/vs*.wav'):
    (fname_val_s.append(filename))
fname_val_s.sort()
# fname_trs

fname_val_x=[]
for filename in glob.glob('/opt/e533/timit-homework/v/vx*.wav'):
    (fname_val_x.append(filename))
fname_val_x.sort()


### Read file names for test data

In [5]:
### File names for Test Data

fname_test=[]
for filename in glob.glob('/opt/e533/timit-homework/te/tex*.wav'):
    (fname_test.append(filename))
fname_test.sort()


### Write data into txt file

In [6]:
### Function For writing file
import librosa
def write_file(file_name, fname_list):
    ### Writing training data S

    with open(file_name, 'wb') as fs:
        for i in range(len(fname_list)):
            sn, sr=librosa.load(fname_list[i], sr=None)
            Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
            mag_Sn=np.abs(Sn)
    #         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
            np.savetxt(fs, mag_Sn, fmt='%.5f')
            fs.write(b'\n')
    fs.close()            

    

### Writing complex data X into file 

In [7]:
### Function For writing just X into file
import librosa
def write_file_X(file_name, fname_list):
    ### Writing training data S

    with open(file_name, 'wb') as fs:
        for i in range(len(fname_list)):
            sn, sr=librosa.load(fname_list[i], sr=None)
            Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
#             mag_Sn=np.abs(Sn)
    #         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
            np.savetxt(fs, Sn, fmt='%.5f')
            fs.write(b'\n')
    fs.close()            

    

In [8]:
### Writing training data S
Train_complx_X=[]

def read_complex_data(fname_trx):
    Train_complx_X=[]
    for i in range(len(fname_trx)):
        print(i),

        sn, sr=librosa.load(fname_trx[i], sr=None)
        Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
        Train_complx_X.append(np.array(Sn))
    return Train_complx_X


#             mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
   


In [9]:
Train_complx_X=read_complex_data(fname_trx)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [10]:
Train_complx_X[0]

array([[-0.3853499 +0.0000000e+00j,  0.060454  +0.0000000e+00j,
         0.01165429+0.0000000e+00j, ...,  0.01285191+0.0000000e+00j,
        -0.21756761+0.0000000e+00j,  0.24008064+0.0000000e+00j],
       [ 0.09418306-8.3754618e-18j, -0.23302463+1.5076146e-01j,
        -0.08883861-2.4753909e-01j, ...,  0.1361649 +2.1456896e-01j,
         0.18064187-2.7819940e-01j, -0.4199345 -7.4127577e-02j],
       [ 0.05248116+2.3452779e-17j,  0.6149999 -1.4774410e-01j,
         0.04419272+5.4086918e-01j, ..., -0.3379148 -4.0866500e-01j,
         0.41280326+2.7587944e-01j,  0.25685748+1.3153780e-01j],
       ...,
       [ 0.1367664 -2.6360976e-18j, -0.05970686-8.4518110e-03j,
         0.00300826+9.1337882e-02j, ..., -0.10093805+2.0822957e-01j,
         0.04817577+1.6439554e-01j, -0.12954259+2.0553760e-01j],
       [ 0.0008341 +1.3145951e-17j,  0.04312928-2.8533509e-02j,
         0.01229421-4.8449092e-02j, ...,  0.04757174-1.0008689e-01j,
         0.01047064-5.0540548e-03j,  0.24539188-1.7348814e-01j]

In [11]:
val_complx_X=read_complex_data(fname_val_x)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [12]:
val_complx_S=read_complex_data(fname_val_s)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [13]:
test_complx_S=read_complex_data(fname_test)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

### Calling functions to write data

In [ ]:
write_file("train_s.txt", fname_trs)
write_file("train_n.txt", fname_trn)
write_file("train_x.txt", fname_trx)

In [6]:
### Write validation files

write_file("validation_s.txt", fname_val_s)
write_file("validation_n.txt", fname_val_n)
write_file("validation_x.txt", fname_val_x)


In [8]:
write_file_X("train_x_tr.txt", fname_trx)
write_file_X("validation_x_tr.txt", fname_trx)


In [8]:
write_file("test_data.txt", fname_test)

In [9]:
### Writing training data N
import librosa

count=0
total_train_s=[]
with open('train_n.txt', 'wb') as fn:
    for i in range(len(fname_trn)):
        sn, sr=librosa.load(fname_trn[i], sr=None)
        Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
        np.savetxt(fn, mag_Sn, fmt='%.5f')
        fn.write(b'\n')
fn.close()            

In [10]:
### Writing training data S

with open('train_s.txt', 'wb') as fs:
    for i in range(len(fname_trs)):
        sn, sr=librosa.load(fname_trs[i], sr=None)
        Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
        np.savetxt(fs, mag_Sn, fmt='%.5f')
        fs.write(b'\n')
fs.close()            


In [11]:
### Writing training file X

with open('train_x.txt', 'wb') as fs:
    for i in range(len(fname_trx)):
        sn, sr=librosa.load(fname_trx[i], sr=None)
        Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
        np.savetxt(fs, mag_Sn, fmt='%.5f')
        fs.write(b'\n')
fs.close()            


### Read data from files

In [18]:
### Function for Reading file

def read_file(file_name):
    with open(file_name) as f:
        lines=f.readlines()
        print(len(lines))
        sentence_full=[]
        count = 0
        sentence=[]
        for line in lines:

            if count < 513:
                if count ==0:
                    sentence=np.array(np.fromstring(line, dtype=float, sep=' '), ndmin=2)
                    count+=1
                else:
                    myarray = np.array(np.fromstring(line, dtype=float, sep=' '), ndmin=2)
                    sentence=np.concatenate((sentence, myarray), axis=0)
                    count+=1
            else:
                sentence_full.append(sentence) 
                count=0
                sentence=[]
        return sentence_full


### Read training data

In [19]:
data_train_n = read_file("train_n.txt")
data_train_s = read_file("train_s.txt")
data_train_x = read_file("train_x.txt")


616800
616800
616800


### Read validation data

In [20]:
data_val_n = read_file("validation_n.txt")
data_val_s = read_file("validation_s.txt")
data_val_x = read_file("validation_x.txt")

616800
616800
616800


### Read complex X from training and validation data

In [23]:
data_train_xtr = read_file("train_x_tr.txt")
data_val_xtr = read_file("validation_x_tr.txt")

616800
616800


In [29]:
len(data_val_xtr)
data_val_xtr[0].shape

(513, 1)

In [21]:
data_test = read_file("test_data.txt")

205600


### Calculating M for training and validation data

In [22]:
### Calculating M:
data_train_M=[]
data_val_M=[]
for i in range(len(data_train_s)):
    data_train_M.append(1*(data_train_s[i]>data_train_n[i]))
    data_val_M.append(1*(data_val_s[i]>data_val_n[i]))
    
    

In [64]:
data_train_M[2].shape

(513, 65)

### Creating Batch  

In [23]:
def next_batchXSCmplx(X_, S_, X_cmplx, S_cmplx):
    
    batch_x = None
    batch_s = None
    batch_x_cmplx = None
    batch_s_cmplx = None
    
    for e,(x, s, x_cmplx, s_cmplx) in enumerate(zip(X_, S_, X_cmplx, S_cmplx)): 
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        batch_s = np.array(s.T) if batch_s is None else np.concatenate( (batch_s,s.T), axis=0)
        batch_x_cmplx = np.array(x_cmplx.T) if batch_x_cmplx is None else np.concatenate( (batch_x_cmplx,x_cmplx.T), axis=0)
        batch_s_cmplx = np.array(s_cmplx.T) if batch_s_cmplx is None else np.concatenate( (batch_s_cmplx,s_cmplx.T), axis=0)
 
        if e>0 and (e+1)%10==0:
            temp_x, batch_x = batch_x, None
            temp_s, batch_s = batch_s, None
            temp_x_cmplx, batch_x_cmplx = batch_x_cmplx, None
            temp_s_cmplx, batch_s_cmplx = batch_s_cmplx, None
            
            temp_x = temp_x.reshape((-1,Max_RNN,513))
            temp_s = temp_s.reshape((-1,Max_RNN,513))
            temp_x_cmplx = temp_x_cmplx.reshape((-1,Max_RNN,513))
#             temp_s_cmplx = temp_s_cmplx.reshape((-1,Max_RNN,513))

            yield temp_x, temp_s, temp_x_cmplx, temp_s_cmplx

In [24]:
def next_batch(X_,Y_):
    
    batch_x, batch_y = None, None
    
    for e,(x,y) in enumerate(zip(X_,Y_)):
#         print(e)
        
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        batch_y = np.array(y.T) if batch_y is None else np.concatenate( (batch_y,y.T), axis=0)
        
#         print('batch_x',batch_x.shape)
#         print('batch_y',batch_y.shape)
        
        if e>0 and (e+1)%10==0:
            temp_x, batch_x = batch_x, None
            temp_y, batch_y = batch_y, None
            
            temp_x = temp_x.reshape((-1,Max_RNN,513))
            temp_y = temp_y.reshape((-1,Max_RNN,513))

#             print('temp_x',temp_x.shape)
#             print('temp_y',temp_y.shape)
        
            yield temp_x,temp_y


### RNN implementation

In [25]:
Max_RNN=5
model = Sequential()

model.add(Bidirectional(GRU(Max_RNN, return_sequences=True), input_shape=(Max_RNN,513)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(Max_RNN, return_sequences=True)))
# model.add(GRU(output_dim = 513, input_length = 5, input_dim = 513, return_sequences=True))

# model.add(Activation('relu'))
# model.add(TimeDistributed(Dense(513, activation='sigmoid')))
model.add(Dense(513, activation='sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

for e in range(10):
    for (b_x,b_y), (v_x,v_y) in zip(next_batch(data_train_x, data_train_M), next_batch(data_val_x, data_val_M)):
        model.fit(b_x, b_y, validation_data=(v_x,v_y), shuffle=True, batch_size=100)

#     model.fit( , epochs=20, steps_per_epoch=700, validation_data=next_batch_mb(DATA_val_x, DATA_val_x,10), validation_steps=700)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 5, 10)             15570     
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 10)             0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 5, 10)             480       
_________________________________________________________________
dense_1 (Dense)              (None, 5, 513)            5643      
Total params: 21,693
Trainable params: 21,693
Non-trainable params: 0
_________________________________________________________________
None
Train on 130 samples, validate on 130 samples
Epoch 1/1
130/130 [==============================] - 3s 22ms/step - loss: 0.6931 - acc: 0.5061 - val_loss: 0.6927 - val_acc: 0.5190
Train on 260 samples, validate on 260 samples
Epoch 1/1
260/260 [======================

Train on 210 samples, validate on 210 samples
Epoch 1/1
210/210 [==============================] - 0s 618us/step - loss: 0.6780 - acc: 0.5568 - val_loss: 0.6727 - val_acc: 0.5555
Train on 158 samples, validate on 158 samples
Epoch 1/1
158/158 [==============================] - 0s 573us/step - loss: 0.6564 - acc: 0.6346 - val_loss: 0.6681 - val_acc: 0.5638
Train on 198 samples, validate on 198 samples
Epoch 1/1
198/198 [==============================] - 0s 553us/step - loss: 0.6554 - acc: 0.6167 - val_loss: 0.6622 - val_acc: 0.5848
Train on 206 samples, validate on 206 samples
Epoch 1/1
206/206 [==============================] - 0s 644us/step - loss: 0.6869 - acc: 0.5014 - val_loss: 0.6725 - val_acc: 0.5514
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 550us/step - loss: 0.6616 - acc: 0.5784 - val_loss: 0.6545 - val_acc: 0.6066
Train on 214 samples, validate on 214 samples
Epoch 1/1
214/214 [==============================] - 0s 641

Train on 228 samples, validate on 228 samples
Epoch 1/1
228/228 [==============================] - 0s 592us/step - loss: 0.6902 - acc: 0.5400 - val_loss: 0.6604 - val_acc: 0.6101
Train on 202 samples, validate on 202 samples
Epoch 1/1
202/202 [==============================] - 0s 669us/step - loss: 0.6510 - acc: 0.6290 - val_loss: 0.6529 - val_acc: 0.6234
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 577us/step - loss: 0.6651 - acc: 0.5727 - val_loss: 0.6542 - val_acc: 0.6240
Train on 242 samples, validate on 242 samples
Epoch 1/1
242/242 [==============================] - 0s 640us/step - loss: 0.6822 - acc: 0.5617 - val_loss: 0.6774 - val_acc: 0.5806
Train on 144 samples, validate on 144 samples
Epoch 1/1
144/144 [==============================] - 0s 656us/step - loss: 0.6466 - acc: 0.6504 - val_loss: 0.6524 - val_acc: 0.6153
Train on 356 samples, validate on 356 samples
Epoch 1/1
356/356 [==============================] - 0s 558

Train on 154 samples, validate on 154 samples
Epoch 1/1
154/154 [==============================] - 0s 573us/step - loss: 0.6194 - acc: 0.6541 - val_loss: 0.6171 - val_acc: 0.6577
Train on 262 samples, validate on 262 samples
Epoch 1/1
262/262 [==============================] - 0s 636us/step - loss: 0.6703 - acc: 0.5984 - val_loss: 0.6349 - val_acc: 0.6457
Train on 156 samples, validate on 156 samples
Epoch 1/1
156/156 [==============================] - 0s 625us/step - loss: 0.5974 - acc: 0.7016 - val_loss: 0.5755 - val_acc: 0.7238
Train on 158 samples, validate on 158 samples
Epoch 1/1
158/158 [==============================] - 0s 614us/step - loss: 0.6445 - acc: 0.6251 - val_loss: 0.6603 - val_acc: 0.6374
Train on 188 samples, validate on 188 samples
Epoch 1/1
188/188 [==============================] - 0s 510us/step - loss: 0.6133 - acc: 0.6768 - val_loss: 0.6060 - val_acc: 0.6920
Train on 216 samples, validate on 216 samples
Epoch 1/1
216/216 [==============================] - 0s 645

Train on 118 samples, validate on 118 samples
Epoch 1/1
118/118 [==============================] - 0s 718us/step - loss: 0.6133 - acc: 0.6660 - val_loss: 0.6231 - val_acc: 0.6619
Train on 232 samples, validate on 232 samples
Epoch 1/1
232/232 [==============================] - 0s 622us/step - loss: 0.6137 - acc: 0.6830 - val_loss: 0.6218 - val_acc: 0.6834
Train on 150 samples, validate on 150 samples
Epoch 1/1
150/150 [==============================] - 0s 587us/step - loss: 0.5640 - acc: 0.7454 - val_loss: 0.5843 - val_acc: 0.7172
Train on 144 samples, validate on 144 samples
Epoch 1/1
144/144 [==============================] - 0s 596us/step - loss: 0.6386 - acc: 0.6455 - val_loss: 0.6200 - val_acc: 0.6673
Train on 156 samples, validate on 156 samples
Epoch 1/1
156/156 [==============================] - 0s 611us/step - loss: 0.5877 - acc: 0.7006 - val_loss: 0.5722 - val_acc: 0.7167
Train on 212 samples, validate on 212 samples
Epoch 1/1
212/212 [==============================] - 0s 627

...

Train on 186 samples, validate on 186 samples
Epoch 1/1
186/186 [==============================] - 0s 481us/step - loss: 0.5675 - acc: 0.7074 - val_loss: 0.5342 - val_acc: 0.7280
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 524us/step - loss: 0.5408 - acc: 0.7351 - val_loss: 0.5299 - val_acc: 0.7406
Train on 222 samples, validate on 222 samples
Epoch 1/1
222/222 [==============================] - 0s 578us/step - loss: 0.5399 - acc: 0.7415 - val_loss: 0.5462 - val_acc: 0.7274
Train on 296 samples, validate on 296 samples
Epoch 1/1
296/296 [==============================] - 0s 466us/step - loss: 0.5467 - acc: 0.7295 - val_loss: 0.5322 - val_acc: 0.7449
Train on 230 samples, validate on 230 samples
Epoch 1/1
230/230 [==============================] - 0s 568us/step - loss: 0.5334 - acc: 0.7375 - val_loss: 0.5335 - val_acc: 0.7364
Train on 190 samples, validate on 190 samples
Epoch 1/1
190/190 [==============================] - 0s 470

Train on 234 samples, validate on 234 samples
Epoch 1/1
234/234 [==============================] - 0s 566us/step - loss: 0.5098 - acc: 0.7538 - val_loss: 0.4728 - val_acc: 0.7820
Train on 184 samples, validate on 184 samples
Epoch 1/1
184/184 [==============================] - 0s 509us/step - loss: 0.5086 - acc: 0.7649 - val_loss: 0.5134 - val_acc: 0.7574
Train on 238 samples, validate on 238 samples
Epoch 1/1
238/238 [==============================] - 0s 574us/step - loss: 0.5550 - acc: 0.7154 - val_loss: 0.5036 - val_acc: 0.7561
Train on 340 samples, validate on 340 samples
Epoch 1/1
340/340 [==============================] - 0s 521us/step - loss: 0.5390 - acc: 0.7331 - val_loss: 0.5150 - val_acc: 0.7532
Train on 220 samples, validate on 220 samples
Epoch 1/1
220/220 [==============================] - 0s 604us/step - loss: 0.5244 - acc: 0.7523 - val_loss: 0.5321 - val_acc: 0.7482
Train on 260 samples, validate on 260 samples
Epoch 1/1
260/260 [==============================] - 0s 511

Train on 176 samples, validate on 176 samples
Epoch 1/1
176/176 [==============================] - 0s 502us/step - loss: 0.5279 - acc: 0.7437 - val_loss: 0.5434 - val_acc: 0.7308
Train on 212 samples, validate on 212 samples
Epoch 1/1
212/212 [==============================] - 0s 595us/step - loss: 0.5294 - acc: 0.7368 - val_loss: 0.5332 - val_acc: 0.7310
Train on 330 samples, validate on 330 samples
Epoch 1/1
330/330 [==============================] - 0s 557us/step - loss: 0.5702 - acc: 0.7000 - val_loss: 0.5779 - val_acc: 0.6949
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 507us/step - loss: 0.5678 - acc: 0.7008 - val_loss: 0.4908 - val_acc: 0.7676
Train on 256 samples, validate on 256 samples
Epoch 1/1
256/256 [==============================] - 0s 542us/step - loss: 0.5466 - acc: 0.7239 - val_loss: 0.5009 - val_acc: 0.7651
Train on 264 samples, validate on 264 samples
Epoch 1/1
264/264 [==============================] - 0s 503

Train on 194 samples, validate on 194 samples
Epoch 1/1
194/194 [==============================] - 0s 482us/step - loss: 0.5266 - acc: 0.7372 - val_loss: 0.5104 - val_acc: 0.7554
Train on 164 samples, validate on 164 samples
Epoch 1/1
164/164 [==============================] - 0s 519us/step - loss: 0.5770 - acc: 0.6914 - val_loss: 0.5487 - val_acc: 0.7243
Train on 346 samples, validate on 346 samples
Epoch 1/1
346/346 [==============================] - 0s 517us/step - loss: 0.5456 - acc: 0.7234 - val_loss: 0.5290 - val_acc: 0.7382
Train on 172 samples, validate on 172 samples
Epoch 1/1
172/172 [==============================] - 0s 518us/step - loss: 0.5323 - acc: 0.7420 - val_loss: 0.5408 - val_acc: 0.7430
Train on 286 samples, validate on 286 samples
Epoch 1/1
286/286 [==============================] - 0s 481us/step - loss: 0.4890 - acc: 0.7742 - val_loss: 0.4947 - val_acc: 0.7629
Train on 186 samples, validate on 186 samples
Epoch 1/1
186/186 [==============================] - 0s 503

Train on 154 samples, validate on 154 samples
Epoch 1/1
154/154 [==============================] - 0s 552us/step - loss: 0.5614 - acc: 0.7126 - val_loss: 0.5449 - val_acc: 0.7350
Train on 166 samples, validate on 166 samples
Epoch 1/1
166/166 [==============================] - 0s 561us/step - loss: 0.5294 - acc: 0.7386 - val_loss: 0.5068 - val_acc: 0.7574
Train on 224 samples, validate on 224 samples
Epoch 1/1
224/224 [==============================] - 0s 595us/step - loss: 0.5166 - acc: 0.7500 - val_loss: 0.4904 - val_acc: 0.7680
Train on 294 samples, validate on 294 samples
Epoch 1/1
294/294 [==============================] - 0s 462us/step - loss: 0.5394 - acc: 0.7346 - val_loss: 0.5218 - val_acc: 0.7464
Train on 150 samples, validate on 150 samples
Epoch 1/1
150/150 [==============================] - 0s 562us/step - loss: 0.4972 - acc: 0.7522 - val_loss: 0.4952 - val_acc: 0.7657
Train on 196 samples, validate on 196 samples
Epoch 1/1
196/196 [==============================] - 0s 456

### Calculating Accuracy

In [26]:
scores = []
for v_x,v_y in next_batch(data_val_x, data_val_M):
    scores.append( model.evaluate(v_x, v_y,verbose=0)[1] )
scores=np.mean(scores)
print("Accuracy: ",  scores*100)

Accuracy:  74.60236812739446


### Calculating SNR

In [27]:
sum_s = 0.0
sum_s_diff = 0.0

for v_s,v_x,v_x_cmplx,v_s_cmplx in next_batchXSCmplx(data_val_s,data_val_x,val_complx_X,val_complx_S):
    
#     print(v_s.shape)
#     print(v_x.shape)
#     print(v_x_cmplx.shape)
#     print('v_s_cmplx',v_s_cmplx.shape)
    
    mask = model.predict(v_x)
    S_hat = (mask) * v_x_cmplx
    S_hat = S_hat.reshape(-1,513).T
    S = v_s_cmplx.T
#     S=S.reshape(-1,513)
    
#     print('S.shape',S.shape)
#     print('S_hat.shape',S_hat.shape)

    S_org = librosa.istft(S, hop_length=512)
    S_pred = librosa.istft(S_hat, hop_length=512)

    sum_s += np.sum(S_org*S_org)
    sum_s_diff += np.sum((S_org-S_pred)*(S_org-S_pred))
    
acc = sum_s/ sum_s_diff
print(acc)

10*np.log10(acc)

3.631143355294444


5.600433949289625

### Write into audio file

In [29]:
def write_audio1(file_name):
    for i in range(len(file_name)):
        audio_fname=file_name[i].replace("/opt/e533/timit-homework/te/", "").replace(".wav","")
        print(audio_fname)
        sn, sr=librosa.load(file_name[i], sr=None)
        Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
        print(mag_Sn.shape)
#         mag_Sn=mag_Sn.reshape(-1, 5, 513)
        Stest_hat=model.predict(mag_Sn.reshape(-1, 5, 513))
        Stest_hat=Stest_hat.reshape(-1,513)
        S_hat=(Sn/mag_Sn)*Stest_hat.T

        S_time=librosa.istft(S_hat, hop_length=512)
        audio_fname=audio_fname + "_recons.wav"
        print(audio_fname)
        librosa.output.write_wav(audio_fname, S_time, sr)
        
        
    

In [41]:
def write_audio(fname_test):
    mags = None
    cmplxs = None
    count=0
    for e, file_x in enumerate(fname_test):
        print(e)
        
        sn, sr = librosa.load(file_x, sr=None)
        Sn = librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
        
        mags = np.array(mag_Sn.T) if mags is None else np.concatenate( (mags,mag_Sn.T), axis=0)
        cmplxs = np.array(Sn.T) if cmplxs is None else np.concatenate( (cmplxs,Sn.T), axis=0)
        
        if e>0 and (e+1)%10==0:
            temp, mags = mags, None
            temp_cmplx, cmplxs = cmplxs, None
            
            temp = temp.reshape((-1,Max_RNN,513))
            mask = model.predict(temp)
            
            mask=mask.reshape(-1,513)
            S_hat = (mask) * temp_cmplx
            S_hat = S_hat.T
            
            lenght_w = S_hat.shape[1]//10
            print(S_hat.shape[1])
            for clip in range(10):
                start_w = clip*lenght_w
                end_w = (clip+1)*lenght_w
                
                wav = S_hat[:,start_w:end_w].T
                S_time=librosa.istft(wav, hop_length=512)
#                 fname = PATH_directory+PATH_denoise+ e + "_redoise.wav"
#                 audio_fname=file_x.replace("/opt/e533/timit-homework/te/", "").replace(".wav","")
                audio_fname="./data/"+ str(count) + "_recons.wav"
                print(audio_fname)
                count+=1
                librosa.output.write_wav(audio_fname, S_time, sr)
                

In [42]:
write_audio(fname_test)

0
1
2
3
4
5
6
7
8
9
1100
./data/0_recons.wav
./data/1_recons.wav
./data/2_recons.wav
./data/3_recons.wav
./data/4_recons.wav
./data/5_recons.wav
./data/6_recons.wav
./data/7_recons.wav
./data/8_recons.wav
./data/9_recons.wav
10
11
12
13
14
15
16
17
18
19
1090
./data/10_recons.wav
./data/11_recons.wav
./data/12_recons.wav
./data/13_recons.wav
./data/14_recons.wav
./data/15_recons.wav
./data/16_recons.wav
./data/17_recons.wav
./data/18_recons.wav
./data/19_recons.wav
20
21
22
23
24
25
26
27
28
29
1730
./data/20_recons.wav
./data/21_recons.wav
./data/22_recons.wav
./data/23_recons.wav
./data/24_recons.wav
./data/25_recons.wav
./data/26_recons.wav
./data/27_recons.wav
./data/28_recons.wav
./data/29_recons.wav
30
31
32
33
34
35
36
37
38
39
1650
./data/30_recons.wav
./data/31_recons.wav
./data/32_recons.wav
./data/33_recons.wav
./data/34_recons.wav
./data/35_recons.wav
./data/36_recons.wav
./data/37_recons.wav
./data/38_recons.wav
./data/39_recons.wav
40
41
42
43
44
45
46
47
48
49
1080
./dat

321
322
323
324
325
326
327
328
329
1610
./data/320_recons.wav
./data/321_recons.wav
./data/322_recons.wav
./data/323_recons.wav
./data/324_recons.wav
./data/325_recons.wav
./data/326_recons.wav
./data/327_recons.wav
./data/328_recons.wav
./data/329_recons.wav
330
331
332
333
334
335
336
337
338
339
1180
./data/330_recons.wav
./data/331_recons.wav
./data/332_recons.wav
./data/333_recons.wav
./data/334_recons.wav
./data/335_recons.wav
./data/336_recons.wav
./data/337_recons.wav
./data/338_recons.wav
./data/339_recons.wav
340
341
342
343
344
345
346
347
348
349
560
./data/340_recons.wav
./data/341_recons.wav
./data/342_recons.wav
./data/343_recons.wav
./data/344_recons.wav
./data/345_recons.wav
./data/346_recons.wav
./data/347_recons.wav
./data/348_recons.wav
./data/349_recons.wav
350
351
352
353
354
355
356
357
358
359
1200
./data/350_recons.wav
./data/351_recons.wav
./data/352_recons.wav
./data/353_recons.wav
./data/354_recons.wav
./data/355_recons.wav
./data/356_recons.wav
./data/357_